### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 3 мая 2021, 08:30   
**Штраф за опоздание:** -2 балла после 08:30 03 мая, -4 балла после 08:30 10 мая, -6 баллов после 08:30 17 мая, -8 баллов после 08:30 24 мая.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0221, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

##  Реализуем дерево решений (3 балла)

Допишите недостающие части дерева решений. Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn.
Внимание: если Вас не устраивает предложенная структура хранения дерева, Вы без потери баллов можете сделать свой класс DecisionTreeClassifier, в котором сами полностью воспроизведете алгоритм дерева решений. Обязательно в нем иметь только функции fit, predict

Код не рабочий, сдаю чтобы понять где ошибка, так как очень хочется разобраться, но ее никак не вижу. Подозреваю что в формуле

In [502]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier


In [922]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=5, criterion='gini'):
        """
        criterion -- критерий расщепления. необходимо релизовать три:
        Ошибка классификации, Индекс Джини, Энтропийный критерий
        max_depth -- максимальная глубина дерева
        min_samples_split -- минимальное число объектов в листе, чтобы сделать новый сплит
        """
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.num_class = -1
        # Для последнего задания
        self.feature_importances_ = None
        self.criterion = criterion
        # Структура, которая описывает дерево
        # Представляет словарь, где для  node_id (айдишник узла дерева) храним
        # (тип_узла, айдишник признака сплита, порог сплита) если тип NON_LEAF_TYPE
        # (тип_узла, предсказание класса, вероятность класса) если тип LEAF_TYPE
        # Подразумевается, что у каждого node_id в дереве слева 
        # узел с айди 2 * node_id + 1, а справа 2 * node_id + 2
        self.tree = dict()

    def __div_samples(self, x, y, feature_id, threshold):
        """
        Разделяет объекты на 2 множества
        x -- матрица объектов
        y -- вектор ответов
        feature_id -- айдишник признака, по которому делаем сплит
        threshold -- порог, по которому делаем сплит
        """
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]
    
    def __gini(self, y):
        p = np.bincount(y)/y.shape
        return (p*(1 - p)).sum()
    
    def __misclass(self, y):
        p = np.bincount(y)/y.shape
        return 1 - np.max(p)
    
    def __entropy(self, y):
        p = np.bincount(y)/y.shape
        return -(p*np.log(p + 0.000001)).sum()
    
        
    def __find_threshold(self, x, y):
        n, samples = x.shape
        top_j, top_val, Q = 0, 0, -1
        
        k1 = np.arange(n)
        k2 = k1[::-1]
        
        if self.criterion == "gini":
            giniFull = np.repeat(self.__gini(y), n)
            
        for j in range(samples):
            a = x[:, j]
            b = np.repeat(a[:, np.newaxis], n, axis=1)
            
            a = x[:, j]
            y_ = y[:, np.newaxis]
            
            A = np.where(b <= a, y_, self.num_class)
            
            N = A.max()+1
            id = A + (N*k1)[None,:]
            p = np.bincount(id.ravel(), minlength=N*n).reshape(-1,N)/n
            n1 = np.bincount(id.ravel(), minlength=N*A.shape[0]).reshape(-1,N)[:,:-1].sum(axis=1)
            l = (p*(1 - p))[:,:-1].sum(axis=1)
            
            A = np.where(b > a, y_, self.num_class)
            N = A.max()+1
            id = A + (N*k1)[None,:]
            p = np.bincount(id.ravel(), minlength=N*n).reshape(-1,N)/n
            n2 = np.bincount(id.ravel(), minlength=N*A.shape[0]).reshape(-1,N)[:,:-1].sum(axis=1)
            r = (p*(1 - p))[:,:-1].sum(axis=1)
            
            gain = giniFull - n1/n*l - n2/n*r
            q_t = np.argmax(gain)
            
            if gain[q_t] > Q:
                Q = gain[q_t]
                top_j = j
                top_val = x[q_t, j]
                
        print(top_j, top_val)
        return top_j, top_val
        
        
    def find_thresholdSlow(self, x, y):
        """
        Находим оптимальный признак и порог для сплита
        Здесь используемые разные impurity в зависимости от self.criterion
        """
        n, samples = x.shape
        top_j, top_val, Q = 0, 0, 0

        if self.criterion == "gini":
            giniFull = self.__gini(y)

        if self.criterion == "misclass":
            misclassFull = self.__misclass(y)

        if self.criterion == "entropy":
            entropyFull = self.__entropy(y)

        for j in range(samples):
            X_col = x[:, j]
            for val in X_col:
                X_l, X_r, y_l, y_r = self.__div_samples(x, y, j, val)
                X_lc, X_rc = X_l.shape[0], X_r.shape[0]
                t_q = 0
                if self.criterion == "gini":
                    t_q = giniFull - X_lc/n*self.__gini(y_l) - X_rc/n*(self.__gini(y_r))

                if self.criterion == "misclass" and y_l.size > 0 and y_r.size > 0:
                    t_q = misclassFull - X_lc/n*self.__misclass(y_l) - X_rc/n*(self.__misclass(y_r))

                if self.criterion == "entropy":
                    t_q = entropyFull - X_lc/n*self.__entropy(y_l) - X_rc/n*(self.__entropy(y_r))
            
                if t_q > Q:
                    Q = t_q
                    top_val = val
                    top_j = j
        print(top_j, top_val)
        return top_j, top_val
    
    def __div_samples(self, x, y, feature_id, threshold):
        """
        Разделяет объекты на 2 множества
        x -- матрица объектов
        y -- вектор ответов
        feature_id -- айдишник признака, по которому делаем сплит
        threshold -- порог, по которому делаем сплит
        """
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __fit_node(self, x, y, node_id, depth):
        """
        Делаем новый узел в дереве
        Решаем, терминальный он или нет
        Если нет, то строим левый узел  с айди 2 * node_id + 1
        И правый узел с  айди 2 * node_id + 2
        """
        if depth == self.max_depth and y.shape[0] != 0:
            a = np.bincount(y)
            t = np.argmax(a)
            self.tree[node_id] = (self.__class__.LEAF_TYPE, t, a[t]/a.sum())
        elif y.shape[0] >= self.min_samples_split:
            a, b = self.find_thresholdSlow(x, y)
            self.tree[node_id] = (self.__class__.NON_LEAF_TYPE, a, b)
            X_l, X_r, y_l, y_r = self.__div_samples(x, y, a, b)
            self.__fit_node(X_l, y_l, 2*node_id + 1, depth + 1)
            self.__fit_node(X_r, y_r, 2*node_id + 2, depth + 1)
        pass
    
    def fit(self, x, y):
        """
        Рекурсивно строим дерево решений
        Начинаем с корня node_id 0
        """
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        """
        Рекурсивно обходим дерево по всем узлам,
        пока не дойдем до терминального
        """
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]
        
    def predict(self, X):
        """
        Вызывает predict для всех объектов из матрицы X
        """
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)
    
    def get_feature_importance():
        """
        Возвращает важность признаков
        """
        # Ваш код здесь
        pass

In [923]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [924]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)
X_test.shape

(18, 13)

In [925]:
my_clf.fit(X_test, y_test)
clf.fit(X_test, y_test)

10 0.6
12 714.0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0


DecisionTreeClassifier()

In [926]:
accuracy_score(y_pred=clf.predict(X_test), y_true=y_test), accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test)

(1.0, 1.0)

## Ускоряем дерево решений (2 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine. 
Для этого используем numpy.

In [703]:
%timeit clf.fit(X_train, y_train)

590 µs ± 3.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [704]:
%timeit my_clf.fit(X_train, y_train)

28.4 ms ± 158 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Боевое применение (3 балла)

На практике Вы познакомились с датасетом Speed Dating Data. В нем каждая пара в быстрых свиданиях характеризуется определенным набором признаков. Задача -- предсказать, произойдет ли матч пары (колонка match). 

Пример работы с датасетом можете найти в практике пункт 2
https://github.com/VVVikulin/ml1.sphere/blob/master/2019-09/lecture_06/pract-trees.ipynb

Данные и описания колонок лежат тут
https://cloud.mail.ru/public/8nHV/p6J7wY1y1/speed-dating-experiment/

Скачайте датасет, обработайте данные, как показано на семинаре или своим собственным способом. Обучите дерево классифкации. В качестве таргета возьмите колонку 'match'. Постарайтесь хорошо обработать признаки, чтобы выбить максимальную точность. Если точность будет близка к случайному гаданию, задание не будет защитано. 


In [901]:
df = pd.read_csv('Speed Dating Data.csv', encoding='latin1')

In [902]:
df.shape

(8378, 195)

In [903]:
df.head()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


In [904]:
df = df.iloc[:, :97]
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df = df.drop(['condtn'], axis=1)
df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['partner'], axis=1)
df = df.drop(['age_o', 'race_o', 'pf_o_att', 
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o','met_o'], 
             axis=1)
df = df.drop(['field'], axis=1)
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)
df = df.drop(['undergra'], axis=1)
df = df.drop(['mn_sat'], axis=1)
df = df.drop(['tuition'], axis=1)
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df = df.dropna(subset=['date'])
df = df.drop(['career'], axis=1)
df = df.drop(['sports','tvsports','exercise','dining','museums','art','hiking','gaming',
       'clubbing','reading','tv','theater','movies','concerts','music','shopping','yoga'], axis=1)
df = df.drop(['expnum'], axis=1)
df = df.drop(['wave'], axis=1)
df = df.dropna(subset=['imprelig', 'imprace'])

df

,iid,gender,order,pid,match,int_corr,samerace,age,field_cd,race,...,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1
0,1,0,4,11.0,0,0.14,0,21.0,1.0,4.0,...,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,0,3,12.0,0,0.54,0,21.0,1.0,4.0,...,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,0,10,13.0,1,0.16,1,21.0,1.0,4.0,...,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,0,5,14.0,1,0.61,0,21.0,1.0,4.0,...,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,0,7,15.0,1,0.21,0,21.0,1.0,4.0,...,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8373,552,1,5,526.0,0,0.64,0,25.0,18.0,2.0,...,8.0,7.0,6.0,7.0,7.0,9.0,7.0,10.0,5.0,9.0
8374,552,1,4,527.0,0,0.71,0,25.0,18.0,2.0,...,8.0,7.0,6.0,7.0,7.0,9.0,7.0,10.0,5.0,9.0
8375,552,1,10,528.0,0,-0.46,0,25.0,18.0,2.0,...,8.0,7.0,6.0,7.0,7.0,9.0,7.0,10.0,5.0,9.0
8376,552,1,16,529.0,0,0.62,0,25.0,18.0,2.0,...,8.0,7.0,6.0,7.0,7.0,9.0,7.0,10.0,5.0,9.0


In [905]:
df = df.dropna(axis='columns')
df

,iid,gender,order,match,samerace,age,field_cd,race,imprace,imprelig,...,date,go_out,exphappy,attr1_1,sinc1_1,intel1_1,attr2_1,sinc2_1,intel2_1,fun2_1
0,1,0,4,0,0,21.0,1.0,4.0,2.0,4.0,...,7.0,1.0,3.0,15.0,20.0,20.0,35.0,20.0,15.0,20.0
1,1,0,3,0,0,21.0,1.0,4.0,2.0,4.0,...,7.0,1.0,3.0,15.0,20.0,20.0,35.0,20.0,15.0,20.0
2,1,0,10,1,1,21.0,1.0,4.0,2.0,4.0,...,7.0,1.0,3.0,15.0,20.0,20.0,35.0,20.0,15.0,20.0
3,1,0,5,1,0,21.0,1.0,4.0,2.0,4.0,...,7.0,1.0,3.0,15.0,20.0,20.0,35.0,20.0,15.0,20.0
4,1,0,7,1,0,21.0,1.0,4.0,2.0,4.0,...,7.0,1.0,3.0,15.0,20.0,20.0,35.0,20.0,15.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8373,552,1,5,0,0,25.0,18.0,2.0,1.0,1.0,...,2.0,1.0,10.0,70.0,0.0,15.0,50.0,0.0,0.0,30.0
8374,552,1,4,0,0,25.0,18.0,2.0,1.0,1.0,...,2.0,1.0,10.0,70.0,0.0,15.0,50.0,0.0,0.0,30.0
8375,552,1,10,0,0,25.0,18.0,2.0,1.0,1.0,...,2.0,1.0,10.0,70.0,0.0,15.0,50.0,0.0,0.0,30.0
8376,552,1,16,0,0,25.0,18.0,2.0,1.0,1.0,...,2.0,1.0,10.0,70.0,0.0,15.0,50.0,0.0,0.0,30.0


Разбейте датасет на трейн и валидацию. Подберите на валидации оптимальный критерий  информативности. 
Постройте графики зависимости точности на валидации от глубины дерева, от минимального числа объектов для сплита. 
Какой максимальной точности удалось достигнуть?

In [906]:
from sklearn.model_selection import KFold

In [907]:
X = df.drop(['match'], axis=1).to_numpy()
y = df.match.to_numpy()
y, X

(array([0, 0, 1, ..., 0, 0, 0]),
 array([[  1.,   0.,   4., ...,  20.,  15.,  20.],
        [  1.,   0.,   3., ...,  20.,  15.,  20.],
        [  1.,   0.,  10., ...,  20.,  15.,  20.],
        ...,
        [552.,   1.,  10., ...,   0.,   0.,  30.],
        [552.,   1.,  16., ...,   0.,   0.,  30.],
        [552.,   1.,  15., ...,   0.,   0.,  30.]]))

In [912]:
N = 3
kf = KFold(n_splits=N)
kf.get_n_splits(X)
print(X.shape)
s = 0
for train_index, test_index in kf.split(X):
    my_clf = DecisionTreeClassifier(min_samples_split=200, max_depth=20, criterion='gini')
    my_clf.fit(X[train_index], y[train_index])
    x = accuracy_score(y_pred=my_clf.predict(X[test_index]), y_true=y[test_index])
    s += x
    print(x)
s/N

(8249, 20)
0.8250909090909091
0.8432727272727273
0.8333939614405238


0.8339191992680534

## Находим самые важные признаки (2 балла)



По построенному дереву  легко понять, какие признаки лучше всего помогли решить задачу. Часто это бывает нужно  не только  для сокращения размерности в данных, но и для лучшего понимания прикладной задачи. Например, Вы хотите понять, какие признаки стоит еще конструировать -- для этого нужно понимать, какие из текущих лучше всего работают в дереве. 

Самый простой метод -- посчитать число сплитов, где использовался данные признак. Это не лучший вариант, так как по признаку который принимает всего 2 значения, но который почти точно разделяет выборку, число сплитов будет очень 1, но при этом признак сам очень хороший. 
В этом задании предлагается для каждого признака считать суммарный gain (в лекции обозначено как Q) при использовании этого признака в сплите. Тогда даже у очень хороших признаков с маленьким число сплитов это значение должно быть довольно высоким.  

Реализовать это довольно просто: создаете словарь номер фичи : суммарный гейн и добавляете в нужную фичу каждый раз, когда используете ее при построении дерева. 

Добавьте функционал, который определяет значения feature importance. Обучите дерево на датасете Speed Dating Data.
Выведите 10 главных фичей по важности.

## Фидбек (бесценно)

* Какие аспекты обучения деревьев решений Вам показались непонятными? Какое место стоит дополнительно объяснить?

### Ваш ответ здесь

* Здесь Вы можете оставить отзыв о этой домашней работе или о всем курсе.

### ВАШ ОТЗЫВ ЗДЕСЬ

